<a href="https://colab.research.google.com/github/AllanKamimura/Pai_ta_ON_python/blob/main/school/SME0340_ODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ## Concentração { run: "auto", form-width: "20%", display-mode: "form" }
#@markdown $A \underset{k_{-1}}{\stackrel{k_1}{\rightleftharpoons}}{B}{\stackrel{k_2}{\rightarrow}} C$
#@markdown ---
#@markdown ### Escolha o Método
autovetor = True #@param {type:"boolean"}
laplace = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Selecione a Aproximação
#@markdown * **Estado Quase Estacionario:** $k_2$ >> $k_{-1}$ > $k_1$ \
#@markdown * **Aproximacao Pre-Equilibrio:** $k_1$ > $k_{-1}$ >> $k_2$ \
#@markdown * **$k_{-1}$ proximo de 0:** $k_1$ $\approx k_2$ >> $k_{-1}$ \

options = "k_-1 proximo de 0" #@param ["Estado Quase Estacionario", "Aproximacao Pre-Equilibrio", "k_-1 proximo de 0"]
#@markdown ---
#@markdown ### Resolução do Gráfico
_range = 5 #@param {type:"slider", min:0.05, max:10, step:0.05}

if options == "Estado Quase Estacionario":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 1
    km1 = 10
    k2 = 1000

if options == "Aproximacao Pre-Equilibrio":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 200
    km1 = 100
    k2 = 1

if options == "k_-1 proximo de 0":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 1
    km1 = 0.0001
    k2 = 1

import sympy

t = sympy.symbols("t")

lamb1 = sympy.symbols("lambda1")
lamb2 = sympy.symbols("lambda2")
lamb3 = sympy.symbols("lambda3")

A = sympy.Function("A")
B = sympy.Function("B")
C = sympy.Function("C")

if laplace | autovetor:
    import plotly.graph_objects as go

    fig = go.Figure(layout_title_text="Gráfico {}: Concetração pelo tempo".format(options))

    if autovetor:
        X = sympy.Matrix([A(t),B(t),C(t)])
        dX = sympy.diff(X, t)

        M = sympy.Matrix([[-k1, km1, 0], [k1, -k2-km1, 0], [0, k2, 0]])

        xh = sympy.solvers.ode.dsolve(dX - M*X, ics = {A(0) : A0, B(0) : B0, C(0) : C0})

        import numpy as np

        def matrix(t_eval):
            return np.array([sympy.lambdify(t, xh[0].rhs, "numpy")(t_eval),
                            sympy.lambdify(t, xh[1].rhs, "numpy")(t_eval),
                            sympy.lambdify(t, xh[2].rhs, "numpy")(t_eval)])

        import pandas as pd

        df = pd.DataFrame(
            map(matrix, np.linspace(0.0001,_range,50)),
            columns = ["A(t)", "B(t)", "C(t)"]
            )
        
        fig.add_trace(go.Scatter(y = df["A(t)"],
                            mode = "lines", 
                            name = "A(t)_Autovetor"))

        fig.add_trace(go.Scatter(y = df["B(t)"],
                            mode = "lines", 
                            name = "B(t)_Autovetor"))

        fig.add_trace(go.Scatter(y = df["C(t)"],
                            mode = "lines", 
                            name = "C(t)_Autovetor"))
        
    if laplace:
        deltaf = 4 * k1 * k2 - (k1 + k2 + km1) ** 2
        kf = (k1 + k2 + km1)
        theta = sympy.sqrt(-deltaf) * t / 2
        phi = - kf * t / 2

        A_dummy = A0 * sympy.exp(phi) * (sympy.cosh(theta) - (-k2-km1+k1) / sympy.sqrt(-deltaf) * sympy.sinh(theta)) + B0 * sympy.exp(phi) * (2 * km1 / sympy.sqrt(-deltaf) * sympy.sinh(theta))
        B_dummy = A0 * sympy.exp(phi) * 2 * k1 / sympy.sqrt(-deltaf) * sympy.sinh(theta) + B0 * sympy.exp(phi) * (sympy.sinh(theta) * (-km1 + k1 - k2) / sympy.sqrt(-deltaf) + sympy.cosh(theta))
        C_dummy = A0 * (1 - sympy.exp(phi) * 
                        (kf / sympy.sqrt(-deltaf) * sympy.sinh(theta) + sympy.cosh(theta))) \
                        + B0 * (1 - \
                                sympy.exp(phi) / (2 * k1) * \
                                (sympy.sinh(theta) * (sympy.sqrt(-deltaf) + kf * (-km1 + k1 - k2) / sympy.sqrt(-deltaf)) \
                                + 2 * k1 * sympy.cosh(theta))) + C0

        xl = sympy.Matrix(
            [sympy.Equality(A(t), A_dummy, evaluate = False),
                sympy.Equality(B(t), B_dummy, evaluate = False),
                sympy.Equality(C(t), C_dummy, evaluate = False),
                ]
        )

        import numpy as np

        def transform(t_eval):
            return np.array([xl[0].rhs.subs(t, t_eval).evalf(),
                            xl[1].rhs.subs(t, t_eval).evalf(),
                            xl[2].rhs.subs(t, t_eval).evalf()])

        import pandas as pd

        df2 = pd.DataFrame(
            map(transform, np.linspace(0.0001,_range,50)),
            columns = ["A(t)", "B(t)", "C(t)"],
            dtype = "float64"
            )
        
        fig.add_trace(go.Scatter(y = df2["A(t)"],
                                 mode = "markers", 
                                 name = "A(t)_Laplace", 
                                 marker_symbol = "diamond",
                                 marker = {"color": "orange"}))

        fig.add_trace(go.Scatter(y = df2["B(t)"],
                            mode = "markers", 
                            name = "B(t)_Laplace",
                            marker_symbol = "diamond",
                            marker = {"color": "forestgreen"}))

        fig.add_trace(go.Scatter(y = df2["C(t)"],
                            mode = "markers", 
                            name = "C(t)_Laplace", 
                            marker_symbol = "diamond",
                            marker = {"color": "firebrick"}))
        
    fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
    fig.update_yaxes(title_text = "Concentração (unidades relativas)")
    fig.show()

    import IPython
    if laplace:
        print("Equações Laplace")
        IPython.display.display_latex(xl)
        print("\n")

    if autovetor:
        print("Equações Autovetores")
        new_xh = sympy.Matrix(xh)
        IPython.display.display_latex(new_xh)
        print("\n")
    

Equações Laplace




Equações Autovetores


In [ ]:
sympy.latex(xl)

'\\left[\\begin{matrix}A{\\left(t \\right)} = \\left(0.0049999375011664 \\sinh{\\left(0.0100001249992286 t \\right)} + \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t}\\\\B{\\left(t \\right)} = 99.9987500233391 e^{- 1.00005 t} \\sinh{\\left(0.0100001249992286 t \\right)}\\\\C{\\left(t \\right)} = - \\left(100.00374996084 \\sinh{\\left(0.0100001249992286 t \\right)} + \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t} + 1\\end{matrix}\\right]'

In [ ]:
#@title ## Velocidade
#@markdown ---
#@markdown ### Escolha o Método
autovetor = True #@param {type:"boolean"}
laplace = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Resolução do Gráfico
_range = 5 #@param {type:"slider", min:0.05, max:10, step:0.05}

if laplace | autovetor:
    import plotly.graph_objects as go

    fig = go.Figure(layout_title_text = "Gráfico {}: Velocidade pelo tempo".format(options))

    if autovetor:
        import numpy as np

        def matrix(t_eval):
            return np.array([sympy.lambdify(t, sympy.diff(xh[0].rhs, t), "numpy")(t_eval),
                            sympy.lambdify(t, sympy.diff(xh[1].rhs, t), "numpy")(t_eval),
                            sympy.lambdify(t, sympy.diff(xh[2].rhs, t), "numpy")(t_eval)])

        import pandas as pd

        ddf = pd.DataFrame(
            map(matrix, np.linspace(0.05,_range,50)),
            columns = ["dA(t)/dt", "dB(t)/dt", "dC(t)/dt"]
            )
        
        fig.add_trace(go.Scatter(y = ddf["dA(t)/dt"],
                            mode = "lines", 
                            name = "dA(t)/dt_Autovetor"))

        fig.add_trace(go.Scatter(y = ddf["dB(t)/dt"],
                            mode = "lines", 
                            name = "dB(t)/dt_Autovetor"))

        fig.add_trace(go.Scatter(y = ddf["dC(t)/dt"],
                            mode = "lines", 
                            name = "dC(t)/dt_Autovetor"))
        
    if laplace:
        import numpy as np

        def transform(t_eval):
            return np.array([sympy.diff(xl[0].rhs, t).subs(t, t_eval).evalf(),
                            sympy.diff(xl[1].rhs, t).subs(t, t_eval).evalf(),
                            sympy.diff(xl[2].rhs, t).subs(t, t_eval).evalf()])

        import pandas as pd

        ddf2 = pd.DataFrame(
            map(transform, np.linspace(0.05,_range,50)),
            columns = ["dA(t)/dt", "dB(t)/dt", "dC(t)/dt"],
            dtype = "float64"
            )
        
        fig.add_trace(go.Scatter(y = ddf2["dA(t)/dt"],
                    mode = "markers", 
                    name = "dA(t)/dt_Laplace",
                    marker_symbol = "diamond",
                    marker = {"color": "orange"}))

        fig.add_trace(go.Scatter(y = ddf2["dB(t)/dt"],
                            mode = "markers", 
                            name = "dB(t)/dt_Laplace",
                            marker_symbol = "diamond",
                            marker = {"color": "forestgreen"}))

        fig.add_trace(go.Scatter(y = ddf2["dC(t)/dt"],
                            mode = "markers", 
                            name = "dC(t)/dt_Laplace",
                            marker_symbol = "diamond",
                            marker = {"color": "firebrick"}))
        
    fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
    fig.update_yaxes(title_text = "Velocidade (unidades relativas)")
    fig.show()

    import IPython
    if laplace:
        print("Equações Laplace")
        dxl = sympy.Matrix([sympy.Equality(sympy.diff(xl[0].lhs, t), sympy.diff(xl[0].rhs, t)),
                            sympy.Equality(sympy.diff(xl[1].lhs, t), sympy.diff(xl[1].rhs, t)),
                            sympy.Equality(sympy.diff(xl[2].lhs, t), sympy.diff(xl[2].rhs, t))])
        IPython.display.display_latex(dxl)
        print("\n")

    if autovetor:
        print("Equações Autovetores")
        new_xh = sympy.Matrix(xh)
        dxh = sympy.Matrix([sympy.Equality(sympy.diff(xh[0].lhs, t), sympy.diff(xh[0].rhs, t)),
                            sympy.Equality(sympy.diff(xh[1].lhs, t), sympy.diff(xh[1].rhs, t)),
                            sympy.Equality(sympy.diff(xh[2].lhs, t), sympy.diff(xh[2].rhs, t))])
        IPython.display.display_latex(dxh)
        print("\n")
    

Equações Laplace




Equações Autovetores


In [ ]:
sympy.latex(dxl)

'\\left[\\begin{matrix}\\frac{d}{d t} A{\\left(t \\right)} = - 1.00005 \\left(0.0049999375011664 \\sinh{\\left(0.0100001249992286 t \\right)} + \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t} + \\left(0.0100001249992286 \\sinh{\\left(0.0100001249992286 t \\right)} + 4.99999999999945 \\cdot 10^{-5} \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t}\\\\\\frac{d}{d t} B{\\left(t \\right)} = - 100.00374996084 e^{- 1.00005 t} \\sinh{\\left(0.0100001249992286 t \\right)} + 1.0 e^{- 1.00005 t} \\cosh{\\left(0.0100001249992286 t \\right)}\\\\\\frac{d}{d t} C{\\left(t \\right)} = - 1.00005 \\left(- 100.00374996084 \\sinh{\\left(0.0100001249992286 t \\right)} - \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t} + \\left(- 0.0100001249992286 \\sinh{\\left(0.0100001249992286 t \\right)} - 1.00005 \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t}\\end{matrix}\\right]'

In [ ]:
#@title ## Aceleração
#@markdown ---
#@markdown ### Escolha o Método
autovetor = True #@param {type:"boolean"}
laplace = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Resolução do Gráfico
_range = 5 #@param {type:"slider", min:0.05, max:10, step:0.05}

if laplace | autovetor:
    import plotly.graph_objects as go

    fig = go.Figure(layout_title_text = "Gráfico {}: Aceleração pelo tempo".format(options))

    if autovetor:
        import numpy as np

        def matrix(t_eval):
            return np.array([sympy.lambdify(t, sympy.diff(xh[0].rhs, t, t), "numpy")(t_eval),
                            sympy.lambdify(t, sympy.diff(xh[1].rhs, t, t), "numpy")(t_eval),
                            sympy.lambdify(t, sympy.diff(xh[2].rhs, t, t), "numpy")(t_eval)])

        import pandas as pd

        dddf = pd.DataFrame(
            map(matrix, np.linspace(0.05,_range,50)),
            columns = ["d²A(t)/dt²", "d²B(t)/dt²", "d²C(t)/dt²"]
            )
        
        fig.add_trace(go.Scatter(y = dddf["d²A(t)/dt²"],
                            mode = "lines", 
                            name = "d²A(t)/dt²_Autovetor"))

        fig.add_trace(go.Scatter(y = dddf["d²B(t)/dt²"],
                            mode = "lines", 
                            name = "d²B(t)/dt²_Autovetor"))

        fig.add_trace(go.Scatter(y = dddf["d²C(t)/dt²"],
                            mode = "lines", 
                            name = "d²C(t)/dt²_Autovetor"))
        
    if laplace:
        import numpy as np

        def transform(t_eval):
            return np.array([sympy.diff(xl[0].rhs, t, t).subs(t, t_eval).evalf(),
                            sympy.diff(xl[1].rhs, t, t).subs(t, t_eval).evalf(),
                            sympy.diff(xl[2].rhs, t, t).subs(t, t_eval).evalf()])

        import pandas as pd

        dddf2 = pd.DataFrame(
            map(transform, np.linspace(0.05,_range,50)),
            columns = ["d²A(t)/dt²", "d²B(t)/dt²", "d²C(t)/dt²"],
            dtype = "float64"
            )
        
        fig.add_trace(go.Scatter(y = dddf2["d²A(t)/dt²"],
                    mode = "markers", 
                    name = "d²A(t)/dt²_Laplace",
                    marker_symbol = "diamond",
                    marker = {"color": "orange"}))

        fig.add_trace(go.Scatter(y = dddf2["d²B(t)/dt²"],
                            mode = "markers", 
                            name = "d²B(t)/dt²_Laplace",
                            marker_symbol = "diamond",
                            marker = {"color": "forestgreen"}))

        fig.add_trace(go.Scatter(y = dddf2["d²C(t)/dt²"],
                            mode = "markers", 
                            name = "d²C(t)/dt²_Laplace",
                            marker_symbol = "diamond",
                            marker = {"color": "firebrick"}))
        
    fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
    fig.update_yaxes(title_text = "Velocidade (unidades relativas)")
    fig.show()

    import IPython
    if laplace:
        print("Equações Laplace")
        ddxl = sympy.Matrix([sympy.Equality(sympy.diff(xl[0].lhs, t, t), sympy.diff(xl[0].rhs, t, t)),
                            sympy.Equality(sympy.diff(xl[1].lhs, t, t), sympy.diff(xl[1].rhs, t, t)),
                            sympy.Equality(sympy.diff(xl[2].lhs, t, t), sympy.diff(xl[2].rhs, t, t))])
        IPython.display.display_latex(ddxl)
        print("\n")

    if autovetor:
        print("Equações Autovetores")
        new_xh = sympy.Matrix(xh)
        ddxh = sympy.Matrix([sympy.Equality(sympy.diff(xh[0].lhs, t, t), sympy.diff(xh[0].rhs, t, t)),
                            sympy.Equality(sympy.diff(xh[1].lhs, t, t), sympy.diff(xh[1].rhs, t, t)),
                            sympy.Equality(sympy.diff(xh[2].lhs, t, t), sympy.diff(xh[2].rhs, t, t))])
        IPython.display.display_latex(ddxh)
        print("\n")
    

Equações Laplace




Equações Autovetores


In [ ]:
sympy.latex(ddxl)

'\\left[\\begin{matrix}\\frac{d^{2}}{d t^{2}} A{\\left(t \\right)} = \\left(- 0.0150003124972907 \\sinh{\\left(0.0100001249992286 t \\right)} + 1.0001 \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t}\\\\\\frac{d^{2}}{d t^{2}} B{\\left(t \\right)} = \\left(100.018750273338 \\sinh{\\left(0.0100001249992286 t \\right)} - 2.0001 \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t}\\\\\\frac{d^{2}}{d t^{2}} C{\\left(t \\right)} = \\left(- 100.00374996084 \\sinh{\\left(0.0100001249992286 t \\right)} + 1.0 \\cosh{\\left(0.0100001249992286 t \\right)}\\right) e^{- 1.00005 t}\\end{matrix}\\right]'

In [ ]:
#@title ## Todos os valores { run: "auto", form-width: "20%", display-mode: "form" }

#@markdown ---
#@markdown ### Escolha o Método
metodo = "laplace" #@param ["autovetor", "laplace"]

import pandas as pd
if metodo == "autovetor":
    show = pd.concat([df,ddf,dddf], axis = 1)

if metodo == "laplace":
    show = pd.concat([df2,ddf2,dddf2], axis = 1)

show

,A(t),B(t),C(t),dA(t)/dt,dB(t)/dt,dC(t)/dt,d²A(t)/dt²,d²B(t)/dt²,d²C(t)/dt²
0,0.999900,0.000100,4.999667e-09,-0.951225,0.903663,0.047561,0.951315,-1.854979,0.903663
1,0.902905,0.092221,4.874208e-03,-0.859818,0.729967,0.129851,0.859891,-1.589858,0.729967
2,0.815319,0.166468,1.821274e-02,-0.777196,0.581309,0.195887,0.777254,-1.358563,0.581309
3,0.736231,0.225442,3.832748e-02,-0.702514,0.454480,0.248034,0.702559,-1.157039,0.454480
4,0.664815,0.271407,6.377821e-02,-0.635009,0.346658,0.288350,0.635043,-0.981702,0.346658
5,0.600327,0.306333,9.334036e-02,-0.573991,0.255362,0.318629,0.574016,-0.829378,0.255362
6,0.542095,0.331928,1.259767e-01,-0.518836,0.178410,0.340427,0.518854,-0.697264,0.178410
7,0.489512,0.349675,1.608127e-01,-0.468982,0.113889,0.355093,0.468994,-0.582883,0.113889
8,0.442030,0.360855,1.971144e-01,-0.423919,0.060121,0.363799,0.423925,-0.484046,0.060121
9,0.399154,0.366576,2.342697e-01,-0.383186,0.015634,0.367553,0.383188,-0.398822,0.015634


In [ ]:
#@title ## Parametros da Reação e Condições iniciais { run: "auto", form-width: "20%", display-mode: "form" }
#@markdown $A \underset{k_{-1}}{\stackrel{k_1}{\rightleftharpoons}}{B}{\stackrel{k_2}{\rightarrow}} C$

#@markdown ---
#@markdown ### Escolha o Método
autovetor = True #@param {type:"boolean"}
laplace = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Condição Inicial
A_0 = 1 #@param {type:"slider", min:0.05, max:1, step:0.05}
B_0 = 0.35 #@param {type:"slider", min:0.00, max:1, step:0.05}
C_0 = 0.05 #@param {type:"slider", min:0.00, max:1, step:0.05}
#@markdown ---
#@markdown ### Constantes de Equilibrio
k1 = 1 #@param {type:"number"}
km1 = 0.001#@param {type:"number"}
k2 = 1 #@param {type:"number"}
#@markdown ---
#@markdown ### Resolução do Gráfico
_range = 5.3 #@param {type:"slider", min:0.05, max:10, step:0.05}

A0 = A_0 / (A_0 + B_0 + C_0)
B0 = B_0 / (A_0 + B_0 + C_0)
C0 = C_0 / (A_0 + B_0 + C_0)

options = "geral"

import sympy

t = sympy.symbols("t")

lamb1 = sympy.symbols("lambda1")
lamb2 = sympy.symbols("lambda2")
lamb3 = sympy.symbols("lambda3")

A = sympy.Function("A")
B = sympy.Function("B")
C = sympy.Function("C")

if laplace | autovetor:
    import plotly.graph_objects as go

    fig = go.Figure(layout_title_text="Gráfico {}: Concetração pelo tempo".format(options))

    if autovetor:
        X = sympy.Matrix([A(t),B(t),C(t)])
        dX = sympy.diff(X, t)

        M = sympy.Matrix([[-k1, km1, 0], [k1, -k2-km1, 0], [0, k2, 0]])

        xh = sympy.solvers.ode.dsolve(dX - M*X, ics = {A(0) : A0, B(0) : B0, C(0) : C0})

        import numpy as np

        def matrix(t_eval):
            return np.array([sympy.lambdify(t, xh[0].rhs, "numpy")(t_eval),
                            sympy.lambdify(t, xh[1].rhs, "numpy")(t_eval),
                            sympy.lambdify(t, xh[2].rhs, "numpy")(t_eval)])

        import pandas as pd

        df = pd.DataFrame(
            map(matrix, np.linspace(0.0001,_range,50)),
            columns = ["A(t)", "B(t)", "C(t)"]
            )
        
        fig.add_trace(go.Scatter(y = df["A(t)"],
                            mode = "lines", 
                            name = "A(t)_Autovetor"))

        fig.add_trace(go.Scatter(y = df["B(t)"],
                            mode = "lines", 
                            name = "B(t)_Autovetor"))

        fig.add_trace(go.Scatter(y = df["C(t)"],
                            mode = "lines", 
                            name = "C(t)_Autovetor"))
        
    if laplace:
        deltaf = 4 * k1 * k2 - (k1 + k2 + km1) ** 2
        kf = (k1 + k2 + km1)
        theta = sympy.sqrt(-deltaf) * t / 2
        phi = - kf * t / 2

        A_dummy = A0 * sympy.exp(phi) * (sympy.cosh(theta) - (-k2-km1+k1) / sympy.sqrt(-deltaf) * sympy.sinh(theta)) + B0 * sympy.exp(phi) * (2 * km1 / sympy.sqrt(-deltaf) * sympy.sinh(theta))
        B_dummy = A0 * sympy.exp(phi) * 2 * k1 / sympy.sqrt(-deltaf) * sympy.sinh(theta) + B0 * sympy.exp(phi) * (sympy.sinh(theta) * (-km1 + k1 - k2) / sympy.sqrt(-deltaf) + sympy.cosh(theta))
        C_dummy = A0 * (1 - sympy.exp(phi) * 
                        (kf / sympy.sqrt(-deltaf) * sympy.sinh(theta) + sympy.cosh(theta))) \
                        + B0 * (1 - \
                                sympy.exp(phi) / (2 * k1) * \
                                (sympy.sinh(theta) * (sympy.sqrt(-deltaf) + kf * (-km1 + k1 - k2) / sympy.sqrt(-deltaf)) \
                                + 2 * k1 * sympy.cosh(theta))) + C0

        xl = sympy.Matrix(
            [sympy.Equality(A(t), A_dummy, evaluate = False),
                sympy.Equality(B(t), B_dummy, evaluate = False),
                sympy.Equality(C(t), C_dummy, evaluate = False),
                ]
        )

        import numpy as np

        def transform(t_eval):
            return np.array([xl[0].rhs.subs(t, t_eval).evalf(),
                            xl[1].rhs.subs(t, t_eval).evalf(),
                            xl[2].rhs.subs(t, t_eval).evalf()])

        import pandas as pd

        df2 = pd.DataFrame(
            map(transform, np.linspace(0.0001,_range,50)),
            columns = ["A(t)", "B(t)", "C(t)"],
            dtype = "float64"
            )
        
        fig.add_trace(go.Scatter(y = df2["A(t)"],
                    mode = "markers", 
                    name = "A(t)_Laplace",
                    marker_symbol = "diamond",
                    marker = {"color": "orange"}))

        fig.add_trace(go.Scatter(y = df2["B(t)"],
                                 mode = "markers", 
                                 name = "B(t)_Laplace",
                                 marker_symbol = "diamond",
                                 marker = {"color": "forestgreen"}))

        fig.add_trace(go.Scatter(y = df2["C(t)"],
                            mode = "markers", 
                            name = "C(t)_Laplace",
                            marker_symbol = "diamond",
                            marker = {"color": "firebrick"}))
        
    fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
    fig.update_yaxes(title_text = "Concentração (unidades relativas)")
    fig.show()

    import IPython
    if laplace:
        print("Equações Laplace")
        IPython.display.display_latex(xl)
        print("\n")

    if autovetor:
        print("Equações Autovetores")
        new_xh = sympy.Matrix(xh)
        IPython.display.display_latex(new_xh)
        print("\n")
    

Equações Laplace




Equações Autovetores
